In [1]:
import re
from collections import Counter

def parse_log_line(log_line):
    regex = r"\[(\S+)\s+(\S+)\]\s+(.+)"
    
    # date, time, action
    return re.findall(regex, log_line)[0]

class Guard:
    def __init__(self, guard_id):
        self.guard_id = int(guard_id)
        self.sleep_time = 0
        self._sleep_freq = Counter()
        self._current_sleep_start_minute = None
        
    def start_sleeping(self, raw_time):
        self._current_sleep_start_minute = self._raw_time_to_minute(raw_time)
    
    def stop_sleeping(self, raw_time):
        wake_minute = self._raw_time_to_minute(raw_time)
        
        self.sleep_time += wake_minute - self._current_sleep_start_minute
        self._sleep_freq.update(range(self._current_sleep_start_minute, wake_minute))
    
    def get_most_common_sleep_minute(self):
        return self._sleep_freq.most_common(1)[0][0]
    
    def get_most_common_sleep_minute_freq(self):
        if self.sleep_time == 0:
            return 0
        
        return self._sleep_freq.most_common(1)[0][1]
    
    def _raw_time_to_minute(self, raw_time):
        return int(raw_time.split(":")[1])
    
def parse_guards_log(raw_log):
    guards = dict()
    current_guard = None
    
    for log_line in sorted(raw_log):
        date, time, action = parse_log_line(log_line)
        
        if action == "falls asleep":
            current_guard.start_sleeping(time)
        elif action == "wakes up":
            current_guard.stop_sleeping(time)
        elif action.startswith("Guard"):
            guard_id = re.search(r"\d+", action).group()
            if guard_id not in guards:
                guards[guard_id] = Guard(guard_id)
            
            current_guard = guards[guard_id]
        else:
            raise Exception(f"Not allowed action: {action}")
            
    return guards.values()
            
def part_1_solution(input_list):
    guards = parse_guards_log(input_list)
    
    searched_guard = sorted(guards, key=lambda g: g.sleep_time, reverse=True)[0]
    
    return searched_guard.guard_id * searched_guard.get_most_common_sleep_minute()
    

In [2]:
test_input="""[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-02 00:40] falls asleep
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-03 00:29] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up"""

assert(part_1_solution(test_input.splitlines()) == 240)
print("Test passed")

Test passed


In [3]:
with open("inputs/Day_04.txt") as f:
    puzzle_input = f.readlines()

In [4]:
print(f"Part 1 solution: {part_1_solution(puzzle_input)}")

Part 1 solution: 39698


In [5]:
def part_2_solution(input_list):
    guards = parse_guards_log(input_list)
    
    searched_guard = sorted(guards,
                            key=lambda g: g.get_most_common_sleep_minute_freq(),
                            reverse=True)[0]
    
    return searched_guard.guard_id * searched_guard.get_most_common_sleep_minute()

In [6]:
assert(part_2_solution(test_input.splitlines()) == 4455)
print("Test passed")

Test passed


In [7]:
print(f"Part 2 solution: {part_2_solution(puzzle_input)}")

Part 2 solution: 14920
